In [5]:
import sys
import os
import copy
import math
import pandas as pd
import numpy as np
from sympy import *

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import clear_output
#from google.colab import output
#output.enable_custom_widget_manager()

In [ ]:
import tensorflow as tf
import keras
from keras import initializers
from keras.layers import Input, Dropout, BatchNormalization, Conv3DTranspose, concatenate, Dense, Conv3D, Flatten, MaxPooling3D
from keras.models import Sequential
import keras.backend as K
tf.keras.utils.set_random_seed(0)

from sklearn.cluster import AgglomerativeClustering

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# TRAINING DATA

paths = [x[0] for x in os.walk('train/')][1:]

In [ ]:
# VALIDATION DATA

validation_paths = [x[0] for x in os.walk('validation/')][1:]

In [ ]:
# TESTING DATA

testing_paths = [x[0] for x in os.walk('test/')][1:]

# Cluster Analysis 
In the following cells we perform cluster analysis to compare hotspots produced by the MD simulation and the predictions made by the CNN. 

In [ ]:
def get_clusters(temp_grid):
  x = []
  y = []
  z = []
  temp = []
  for i in range(16):
    for j in range(32):
      for k in range(32):
        if temp_grid[i,j,k] >= 1.8: #temp cut-off
          x.append(i+1)
          y.append(j+1)
          z.append(k+1)
          temp.append(temp_grid[i,j,k])
  
  df = pd.DataFrame(temp,columns=['Temperature'])
  df['X'] = x
  df['Y'] = y
  df['Z'] = z

  pos = df[['X','Y','Z']].values
  agglo = AgglomerativeClustering(n_clusters=None, distance_threshold=1.8, linkage='single').fit(pos)
  df['Cluster ID'] = agglo.labels_

  return df

In [ ]:
def analyze_clusters(df):
  dataframe = df.copy()
  vols = []
  mean_temps = []
  std_temps = []
  cms = []
  mis = []
  #pa1 = []
  #pa2 = []
  #pa3 = []
  ids = []
  #filt = dataframe[dataframe['New ID']<100].dropna()
  for i in range(int(dataframe['Cluster ID'].max())+1):
    if i in dataframe['Cluster ID'].tolist():
      tmp = dataframe[dataframe['Cluster ID'] == i]
      v = len(tmp)*4
      mean = np.mean(tmp['Temperature']*1000)
      std = np.std(tmp['Temperature']*1000)
      m = len(tmp)
      cm_x = tmp['X'].sum()/m
      cm_y = tmp['Y'].sum()/m
      cm_z = tmp['Z'].sum()/m
      cm = np.array([cm_x,cm_y,cm_z])
      r_i = tmp[['X','Y','Z']].values
      i_xx = 0
      i_yy = 0
      i_zz = 0
      i_xy = 0
      i_xz = 0
      i_yz = 0
      for j in range(len(tmp)):
        i_xx += (r_i[j][1]-cm_y)**2 + (r_i[j][2]-cm_z)**2
        i_yy += (r_i[j][0]-cm_x)**2 + (r_i[j][2]-cm_z)**2
        i_zz += (r_i[j][0]-cm_x)**2 + (r_i[j][1]-cm_y)**2

        i_xy += -(r_i[j][0]-cm_x)*(r_i[j][1]-cm_y)
        i_xz += -(r_i[j][0]-cm_x)*(r_i[j][2]-cm_z)
        i_yz += -(r_i[j][1]-cm_y)*(r_i[j][2]-cm_z)

      i_xx = np.around(i_xx,decimals=3)
      i_yy = np.around(i_yy,decimals=3)
      i_zz = np.around(i_zz,decimals=3)
      i_xy = np.around(i_xy,decimals=3)
      i_xz = np.around(i_xz,decimals=3)
      i_yz = np.around(i_yz,decimals=3)

      i_tensor = Matrix([[i_xx,i_xy,i_xz],[i_xy,i_yy,i_yz],[i_xz,i_yz,i_zz]])
      #try:
      i_diag = i_tensor.diagonalize()[1]
      #except:
        #i_diag = 'Diagonalization failed'
      vols.append(v)
      mean_temps.append(mean)
      std_temps.append(std)
      cms.append(cm)
      mis.append(i_diag)
      ids.append(i)
    else:
      pass
  
  df_clust = pd.DataFrame(ids, columns=['cluster id'])
  df_clust['volume'] = vols
  df_clust['mean T (K)'] = mean_temps
  df_clust['std T (K)'] = std_temps
  df_clust['R center of mass'] = cms
  df_clust['moment of inertia'] = mis
  return df_clust

In [ ]:
def plot_clusters(md,pred):
  fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scatter3d"},{"type": "scatter3d"}]],
                    subplot_titles=["MD Clusters","CNN Clusters"], horizontal_spacing = 0.1, vertical_spacing = 0.1)
  fig.update_layout(autosize=False, width=1200, height=800) 
  trace_1 = go.Scatter3d(x = md['X'], y = md['Y'], z=md['Z'], hovertemplate = 'Cluster ID: %{marker.color:.2f}<extra></extra>',
                      mode='markers',  marker=dict(symbol='square', colorscale='rainbow', color = md['Cluster ID']), showlegend=False)
  fig.add_trace(trace_1, row=1, col=1)
  trace_2 = go.Scatter3d(x = pred['X'], y = pred['Y'], z=pred['Z'], hovertemplate = 'Cluster ID: %{marker.color:.2f}<extra></extra>',
                      mode='markers',  marker=dict(symbol='square', colorscale='rainbow', color = pred['Cluster ID']), showlegend=False)
  fig.add_trace(trace_2, row=1, col=2)

  return fig

In [ ]:
md_temp = np.load(str(validation_paths[1])+'/output.npy')/1000
pred_temp = np.load('results/'+str(validation_paths[1])+'/prediction.npy')

md_1 = get_clusters(md_temp)
pred_1 = get_clusters(pred_temp)

md_clust = analyze_clusters(md_1)
pred_clust = analyze_clusters(pred_1)

In [ ]:
plot_clusters(md_1,pred_1)

In [ ]:
def visualize(md_1, pred_1, md_clust, pred_clust):
  fig = make_subplots(rows=1, cols=3, specs=[[{"type": "scatter3d"},{"type": "scatter3d"},{"type": "scatter"}]],
                      subplot_titles=["MD Clusters","CNN Clusters","Temp vs Volume"], horizontal_spacing = 0.1, vertical_spacing = 0.1)
  fig.update_layout(autosize=False, width=1800, height=600) 


  ## MD SIMULATION CLUSTERS ##
  tmp = md_1
  trace_1 = go.Scatter3d(x = tmp['X'], y = tmp['Y'], z = tmp['Z'],text=tmp['Cluster ID'],
                        mode='markers', marker=dict(size=8, line=dict(width=0), symbol='square', colorscale='rainbow', color = tmp['Cluster ID']), showlegend=False, scene='scene1')
 
  fig.add_trace(trace_1, row=1, col=1)

  ## CNN PREDICTED CLUSTERS ##
  tmp2 = pred_1
  trace_4 = go.Scatter3d(x = tmp2['X'], y = tmp2['Y'], z = tmp2['Z'],text=tmp2['Cluster ID'],
                        mode='markers', marker=dict(size=8, line=dict(width=0), symbol='square', colorscale='rainbow', color = tmp2['Cluster ID']), showlegend=False, scene='scene2')


  fig.add_trace(trace_4, row=1, col=2)



  ### Hotspot Temperature vs Volume ###
  trace_7 = go.Scatter(x=md_clust['volume'],y=md_clust['mean T (K)'],error_y=dict(type='data', array=md_clust['std T (K)'],visible=True),text=md_clust['cluster id'],
                      mode='markers',marker=dict(symbol='square', color = 'green',size=18),name='MD')
  trace_8 = go.Scatter(x=pred_clust['volume'],y=pred_clust['mean T (K)'],error_y=dict(type='data', array=pred_clust['std T (K)'],visible=True),text=md_clust['cluster id'],
                        mode='markers',marker=dict(symbol='circle', color = 'red',size=18),name='CNN')

  fig.add_trace(trace_7, row=1, col=3)
  fig.add_trace(trace_8, row=1, col=3)
  fig.update_xaxes(title="Volume")
  fig.update_yaxes(title="Temperature (K)")

  ### Hotspot Moment inertia parity plot ###
  #trace_9 = go.Scatter(x=np.log10(md_clust['moment of inertia']),y=np.log10(pred_clust['moment of inertia']),
   #                   mode='markers',marker=dict(symbol='square',color='green',size=14), showlegend=False)

  #fig.add_trace(trace_9, row=2, col=2)
  #fig.update_yaxes(title="Predicted log(moment of inertia)")
  #fig.update_xaxes(title="MD Simulated log(moment of inertia))")

  #def cam_change(layout, camera):
  #   fig.layout.scene2.camera = camera
  #fig.layout.scene1.on_change(cam_change, 'camera')
  return fig

In [ ]:
visualize(md_1,pred_1,md_clust,pred_clust)

In [ ]:
# CLUSTER ANALYSIS TRAINING DATA

for i, title in enumerate(paths):
  print(i)
  md_temp = np.load(str(title)+'/output.npy')/1000
  pred_temp = np.load('results/'+str(title)+'/prediction.npy')

  md_1 = get_clusters(md_temp)
  pred_1 = get_clusters(pred_temp)

  #md_corr, pred_corr, over = overlap(md_1,pred_1)
  md_clust = analyze_clusters(md_1)
  pred_clust = analyze_clusters(pred_1)

  fig = visualize(md_1,pred_1,md_clust,pred_clust)

  fig.write_html('results/'+str(title)+'/cluster_1.8.html')

In [ ]:
# CLUSTER ANALYSIS VALIDATION DATA

for i, title in enumerate(validation_paths):
  print(i)
  md_temp = np.load(str(title)+'/output.npy')/1000
  pred_temp = np.load('results/'+str(title)+'/prediction.npy')

  md_1 = get_clusters(md_temp)
  pred_1 = get_clusters(pred_temp)

  #md_corr, pred_corr, over = overlap(md_1,pred_1)
  md_clust = analyze_clusters(md_1)
  pred_clust = analyze_clusters(pred_1)

  fig = visualize(md_1,pred_1,md_clust,pred_clust)

  fig.write_html('results/'+str(title)+'/cluster_1.8.html')

In [ ]:
# CLUSTER ANALYSIS TESTING DATA

for i, title in enumerate(testing_paths):
  print(i)
  md_temp = np.load(str(title)+'/output.npy')/1000
  pred_temp = np.load('results/'+str(title)+'/prediction.npy')

  md_1 = get_clusters(md_temp)
  pred_1 = get_clusters(pred_temp)

  #md_corr, pred_corr, over = overlap(md_1,pred_1)
  md_clust = analyze_clusters(md_1)
  pred_clust = analyze_clusters(pred_1)

  fig = visualize(md_1,pred_1,md_clust,pred_clust)

  fig.write_html('results/'+str(title)+'/cluster_1.8.html')

Calculate the overlap between clusters in the MD and clusters in the CNN. (Under development)

In [ ]:
def get_overlap(df1,df2):
  md = df1.copy()
  pred = df2.copy()
  overlap_data = []
  for i in range(md['Cluster ID'].max()+1):
    for j in range(pred['Cluster ID'].max()+1):
      md_tmp = md[md['Cluster ID'] == i]
      pred_tmp = pred[pred['Cluster ID'] == j]
      cnn_over = [v for v in md_tmp[['X','Y','Z']].values.tolist() if v in pred_tmp[['X','Y','Z']].values.tolist()]
      cnn_ratio = len(cnn_over)/len(pred_tmp)
      md_over = [v for v in pred_tmp[['X','Y','Z']].values.tolist() if v in md_tmp[['X','Y','Z']].values.tolist()]
      md_ratio = len(md_over)/len(md_tmp)
      overlap_data.append([i,j,md_ratio,cnn_ratio])
  df_over = pd.DataFrame(overlap_data,columns=['MD id','Pred id','MD ratio','CNN ratio'])
  df_over_ = df_over[df_over['MD ratio']>0].reset_index(drop=True)
  df_md_over = df_over_[df_over_.duplicated(subset=['MD id'],keep=False)].copy()#.reset_index(drop=True)
  df_cnn_over = df_over_[df_over_.duplicated(subset=['Pred id'],keep=False)].copy()#.reset_index(drop=True)
  df_cnn_over = df_cnn_over.sort_values(by='Pred id')
  indx = list(df_md_over.index)
  for i in list(df_cnn_over.index):
    if i not in indx:
      indx.append(i)
  df_single_over = df_over_.drop(indx)
  df_single_over['New id'] = df_single_over['MD id']
  n_id = []
  for id in df_md_over['MD id'].unique():
    tmp = df_md_over[df_md_over['MD id']==id].reset_index(drop=True)
    combined_ratio = 0
    for n in range(len(tmp)):
      combined_ratio+=tmp.iloc[n][2]
    if combined_ratio > 0.333:
      for l in range(len(tmp)):
        n_id.append(id)
    else:
      for l in range(len(tmp)):
        n_id.append(None)
  df_md_over['New id'] = n_id
  n_id = []
  for i,id in enumerate(df_cnn_over['Pred id'].unique()):
    tmp = df_cnn_over[df_cnn_over['Pred id']==id].reset_index(drop=True)
    combined_ratio = 0
    for n in range(len(tmp)):
      combined_ratio+=tmp.iloc[n][3]
    if id in df_md_over['Pred id'].unique():
      for l in range(len(tmp)):
        tmp2 = df_md_over[df_md_over['Pred id']==id].reset_index(drop=True)
        n_id.append(tmp2['New id'][0])
    elif combined_ratio > 0.333:
      for l in range(len(tmp)):
        n_id.append(max(df_md_over['New id'].max(),df_single_over['New id'].max())+i+1)
    else:
      for l in range(len(tmp)):
        n_id.append(None)
  df_cnn_over['New id'] = n_id
  df_over_new = pd.concat([df_single_over,df_md_over,df_cnn_over]).drop_duplicates(subset=['MD id','Pred id']).reset_index(drop=True)
  nan_ls = [None]*len(md)
  md['New ID'] = nan_ls
  for m_id in df_over_new['MD id'].unique():
    for i in range(len(md)):
      if md['Cluster ID'][i]==m_id:
        tmp = df_over_new[df_over_new['MD id']==m_id].copy()
        md['New ID'][i] = tmp.copy()['New id'].iloc[0]
  nan_ls = [None]*len(pred)
  pred['New ID'] = nan_ls
  for p_id in df_over_new['Pred id'].unique():
    for i in range(len(pred)):
      if pred['Cluster ID'][i]==p_id:
        tmp = df_over_new[df_over_new['Pred id']==p_id].copy()
        pred['New ID'][i] = tmp.copy()['New id'].iloc[0] 

  return md, pred, df_over     

In [ ]:
def analyze_clusters_overlap(df):
  dataframe = df.copy()
  vols = []
  mean_temps = []
  std_temps = []
  cms = []
  mis = []
  #pa1 = []
  #pa2 = []
  #pa3 = []
  ids = []
  min_mi = []
  max_mi = []
  mean_mi = []
  filt = dataframe.dropna()
  for i in range(int(filt['New ID'].max())+1):
    if i in filt['New ID'].tolist():
      tmp = filt[filt['New ID'] == i]
      v = len(tmp)*4
      mean = np.mean(tmp['Temperature']*1000)
      std = np.std(tmp['Temperature']*1000)
      m = len(tmp)
      cm_x = tmp['X'].sum()/m
      cm_y = tmp['Y'].sum()/m
      cm_z = tmp['Z'].sum()/m
      cm = np.array([cm_x,cm_y,cm_z])
      r_i = tmp[['X','Y','Z']].values
      i_xx = 0
      i_yy = 0
      i_zz = 0
      i_xy = 0
      i_xz = 0
      i_yz = 0
      for j in range(len(tmp)):
        i_xx += (r_i[j][1]-cm_y)**2 + (r_i[j][2]-cm_z)**2
        i_yy += (r_i[j][0]-cm_x)**2 + (r_i[j][2]-cm_z)**2
        i_zz += (r_i[j][0]-cm_x)**2 + (r_i[j][1]-cm_y)**2

        i_xy += -(r_i[j][0]-cm_x)*(r_i[j][1]-cm_y)
        i_xz += -(r_i[j][0]-cm_x)*(r_i[j][2]-cm_z)
        i_yz += -(r_i[j][1]-cm_y)*(r_i[j][2]-cm_z)

      i_xx = np.around(i_xx,decimals=3)
      i_yy = np.around(i_yy,decimals=3)
      i_zz = np.around(i_zz,decimals=3)
      i_xy = np.around(i_xy,decimals=3)
      i_xz = np.around(i_xz,decimals=3)
      i_yz = np.around(i_yz,decimals=3)

      i_tensor = Matrix([[i_xx,i_xy,i_xz],[i_xy,i_yy,i_yz],[i_xz,i_yz,i_zz]])
      #try:
      i_diag = i_tensor.diagonalize()[1]
      #except:
        #i_diag = 'Diagonalization failed'
      vols.append(v)
      mean_temps.append(mean)
      std_temps.append(std)
      cms.append(cm)
      mis.append(i_diag)
      min_mi.append(float(min(i_diag[0],i_diag[4],i_diag[8])))
      max_mi.append(float(max(i_diag[0],i_diag[4],i_diag[8])))
      mean_mi.append(float(np.mean([i_diag[0],i_diag[4],i_diag[8]])))
      ids.append(i)
    else:
      pass
  
  df_clust = pd.DataFrame(ids, columns=['cluster id'])
  df_clust['volume'] = vols
  df_clust['mean T (K)'] = mean_temps
  df_clust['std T (K)'] = std_temps
  df_clust['R center of mass'] = cms
  df_clust['moment of inertia'] = mis
  df_clust['min principal MI'] = min_mi
  df_clust['max principal MI'] = max_mi
  df_clust['mean principal MI'] = mean_mi
  return df_clust

In [ ]:
def visualize_overlap(md_corr, pred_corr, md_clust, pred_clust):
  fig = make_subplots(rows=2, cols=2, specs=[[{"type": "scatter3d"},{"type": "scatter3d"}],
                                            [{"type": "scatter"},{"type": "scatter"}]],
                      subplot_titles=["MD Clusters","CNN Clusters","Temp vs Volume","Temp vs Principal"], horizontal_spacing = 0.1, vertical_spacing = 0.1)
  fig.update_layout(autosize=False, width=1200, height=800) 


  ## MD SIMULATION CLUSTERS ##
  tmp = md_corr.dropna()
  trace_1 = go.Scatter3d(x = tmp['X'], y = tmp['Y'], z = tmp['Z'],text=tmp['New ID'],
                        mode='markers', marker=dict(size=8, line=dict(width=0), symbol='square', colorscale='rainbow', color = tmp['New ID']), showlegend=False, scene='scene1')
  
  tmp = md_corr[md_corr['New ID'].isnull()]
  trace_2 = go.Scatter3d(x = tmp['X'], y = tmp['Y'], z = tmp['Z'],text=tmp['Cluster ID'],
                        mode='markers', marker=dict(size=8, line=dict(width=0), symbol='square', color = 'black'), showlegend=False, scene='scene1')    
  fig.add_trace(trace_1, row=1, col=1)
  fig.add_trace(trace_2, row=1, col=1)

  ## CNN PREDICTED CLUSTERS ##
  tmp2 = pred_corr.dropna()
  trace_3 = go.Scatter3d(x = tmp2['X'], y = tmp2['Y'], z = tmp2['Z'],text=tmp2['New ID'],
                        mode='markers', marker=dict(size=8, line=dict(width=0), symbol='square', colorscale='rainbow', color = tmp2['New ID']), showlegend=False, scene='scene2')
  
  tmp2 = pred_corr[pred_corr['New ID'].isnull()]
  trace_4 = go.Scatter3d(x = tmp2['X'], y = tmp2['Y'], z = tmp2['Z'],text=tmp2['Cluster ID'],
                        mode='markers', marker=dict(size=8, line=dict(width=0), symbol='square', color = 'black'), showlegend=False, scene='scene2')

  fig.add_trace(trace_3, row=1, col=2)
  fig.add_trace(trace_4, row=1, col=2)


  ### Hotspot Temperature vs Volume ###
  trace_5 = go.Scatter(x=md_clust['volume'],y=md_clust['mean T (K)'],error_y=dict(type='data', array=md_clust['std T (K)'],visible=True),text=md_clust['cluster id'],
                      mode='markers',marker=dict(symbol='square',colorscale='rainbow', color = md_clust['cluster id'],size=18),name='MD')
  trace_6 = go.Scatter(x=pred_clust['volume'],y=pred_clust['mean T (K)'],error_y=dict(type='data', array=pred_clust['std T (K)'],visible=True),text=md_clust['cluster id'],
                        mode='markers',marker=dict(symbol='circle',colorscale='rainbow', color = md_clust['cluster id'],size=18),name='CNN')

  fig.add_trace(trace_5, row=2, col=1)
  fig.add_trace(trace_6, row=2, col=1)
  fig.update_xaxes(title="Volume", row=2, col=1)
  fig.update_yaxes(title="Temperature (K)", row=2, col=1)

  ### Hotspot Moment Inertia vs Temperature ###
  trace_7 = go.Scatter(x=md_clust['min principal MI'],y=md_clust['mean T (K)'],error_y=dict(type='data', array=md_clust['std T (K)'],visible=True),text=md_clust['cluster id'],
                      mode='markers',marker=dict(symbol='square',color='green',size=14), showlegend=False,name='MD min principal axis')
  
  trace_8 = go.Scatter(x=pred_clust['min principal MI'],y=pred_clust['mean T (K)'],error_y=dict(type='data', array=pred_clust['std T (K)'],visible=True),text=pred_clust['cluster id'],
                      mode='markers',marker=dict(symbol='circle',color='green',size=14), showlegend=False,name='CNN min principal axis')
  
  trace_9 = go.Scatter(x=md_clust['max principal MI'],y=md_clust['mean T (K)'],error_y=dict(type='data', array=md_clust['std T (K)'],visible=True),text=md_clust['cluster id'],
                      mode='markers',marker=dict(symbol='square',color='red',size=14), showlegend=False,name='MD max principal axis')
  
  trace_10 = go.Scatter(x=pred_clust['max principal MI'],y=pred_clust['mean T (K)'],error_y=dict(type='data', array=pred_clust['std T (K)'],visible=True),text=pred_clust['cluster id'],
                      mode='markers',marker=dict(symbol='circle',color='red',size=14), showlegend=False,name='CNN max principal axis')
  
  trace_11 = go.Scatter(x=md_clust['mean principal MI'],y=md_clust['mean T (K)'],error_y=dict(type='data', array=md_clust['std T (K)'],visible=True),text=md_clust['cluster id'],
                      mode='markers',marker=dict(symbol='square',color='blue',size=14), showlegend=False,name='MD mean principal axis')
  
  trace_12 = go.Scatter(x=pred_clust['mean principal MI'],y=pred_clust['mean T (K)'],error_y=dict(type='data', array=pred_clust['std T (K)'],visible=True),text=pred_clust['cluster id'],
                      mode='markers',marker=dict(symbol='circle',color='blue',size=14), showlegend=False,name='CNN mean principal axis')
  
  fig.add_trace(trace_7, row=2, col=2)
  fig.add_trace(trace_8, row=2, col=2)
  fig.add_trace(trace_9, row=2, col=2)
  fig.add_trace(trace_10, row=2, col=2)
  fig.add_trace(trace_11, row=2, col=2)
  fig.add_trace(trace_12, row=2, col=2)
  fig.update_xaxes(title="Principal Moment of Inertia", row=2, col=2)
  fig.update_yaxes(title="Temperature (K)", row=2, col=2)


  #def cam_change(layout, camera):
  #   fig.layout.scene2.camera = camera
  #fig.layout.scene1.on_change(cam_change, 'camera')
  return fig

In [ ]:
md_temp = np.load(str(testing_paths[0])+'/output.npy')/1000
pred_temp = np.load('results/'+str(testing_paths[0])+'/prediction.npy')

md_1 = get_clusters(md_temp)
pred_1 = get_clusters(pred_temp)

md_clust = analyze_clusters(md_1)
pred_clust = analyze_clusters(pred_1)

md_1_over, pred_1_over, df_1_over = get_overlap(md_1,pred_1)

md_clust_over = analyze_clusters_overlap(md_1_over)
pred_clust_over = analyze_clusters_overlap(pred_1_over)

test_fig = visualize_overlap(md_1_over,pred_1_over,md_clust_over,pred_clust_over)

In [ ]:
test_fig.write_html('results/'+testing_paths[0]+'/overlap_cluster_1.8.html')

In [ ]:
under_locs = md_1_over[md_1_over['New ID'].isnull()][['X','Y','Z']].values
under_pred = md_1_over[md_1_over['New ID'].isnull()]
over_locs = pred_1_over[pred_1_over['New ID'].isnull()][['X','Y','Z']].values
over_pred = pred_1_over[pred_1_over['New ID'].isnull()]

In [ ]:
test_inputs = np.load('test/smallPBX_4_mirror_btm/input.npy')
test_inputs.shape

In [ ]:
input1 = test_inputs[:,:,:,0]
input2 = test_inputs[:,:,:,1]
input3 = test_inputs[:,:,:,2]

In [ ]:
input1.shape

In [ ]:
input1_vals = []
for i in range(len(under_locs)):
  input1_vals.append(input1[under_locs[i][0]-1,under_locs[i][1]-1,under_locs[i][2]-1])

input2_vals = []
for i in range(len(under_locs)):
  input2_vals.append(input2[under_locs[i][0]-1,under_locs[i][1]-1,under_locs[i][2]-1])

input3_vals = []
for i in range(len(under_locs)):
  input3_vals.append(input3[under_locs[i][0]-1,under_locs[i][1]-1,under_locs[i][2]-1])

In [ ]:
under_pred['Input 1'] = input1_vals
under_pred['Input 2'] = input2_vals
under_pred['Input 3'] = input3_vals

In [ ]:
fig = make_subplots(rows=1, cols=3, specs=[[{"type": "scatter3d"},{"type": "scatter3d"},{"type": "scatter3d"}]],
                    subplot_titles=["Input 1","Input 2","Input 3"], horizontal_spacing = 0.1, vertical_spacing = 0.1)
fig.update_layout(autosize=False, width=1800, height=600) 


## Input 1 ##
trace_1 = go.Scatter3d(x = under_pred['X'], y = under_pred['Y'], z = under_pred['Z'],mode='markers', 
                       marker=dict(colorscale='Viridis',size=8,line=dict(width=0),symbol='square',color=under_pred['Input 1'],colorbar=dict(thickness=20,len=0.3,x=0.27,y=0.9)),
                       showlegend=False)
fig.add_trace(trace_1, row=1, col=1)

## Input 2 ##
trace_2 = go.Scatter3d(x = under_pred['X'], y = under_pred['Y'], z = under_pred['Z'],mode='markers', 
                       marker=dict(colorscale='Viridis',size=8,line=dict(width=0),symbol='square',color=under_pred['Input 2'],colorbar=dict(thickness=20,len=0.3,x=0.62,y=0.9)),
                       showlegend=False)
fig.add_trace(trace_2, row=1, col=2)

## Input 3 ##
trace_3 = go.Scatter3d(x = under_pred['X'], y = under_pred['Y'], z = under_pred['Z'],mode='markers', 
                       marker=dict(colorscale='Viridis',size=8,line=dict(width=0),symbol='square',color=under_pred['Input 3'],colorbar=dict(thickness=20,len=0.3,x=1.0,y=0.9)),
                       showlegend=False)

fig.add_trace(trace_3, row=1, col=3)


In [ ]:
input1_vals = []
for i in range(len(over_locs)):
  input1_vals.append(input1[over_locs[i][0]-1,over_locs[i][1]-1,over_locs[i][2]-1])

input2_vals = []
for i in range(len(over_locs)):
  input2_vals.append(input2[over_locs[i][0]-1,over_locs[i][1]-1,over_locs[i][2]-1])

input3_vals = []
for i in range(len(over_locs)):
  input3_vals.append(input3[over_locs[i][0]-1,over_locs[i][1]-1,over_locs[i][2]-1])

In [ ]:
over_pred['Input 1'] = input1_vals
over_pred['Input 2'] = input2_vals
over_pred['Input 3'] = input3_vals

In [ ]:
fig = make_subplots(rows=1, cols=3, specs=[[{"type": "scatter3d"},{"type": "scatter3d"},{"type": "scatter3d"}]],
                    subplot_titles=["Input 1","Input 2","Input 3"], horizontal_spacing = 0.1, vertical_spacing = 0.1)
fig.update_layout(autosize=False, width=1700, height=600) 


## Input 1 ##
trace_1 = go.Scatter3d(x = over_pred['X'], y = over_pred['Y'], z = over_pred['Z'],mode='markers', 
                       marker=dict(colorscale='Viridis',size=8,line=dict(width=0),symbol='square',color=over_pred['Input 1'],colorbar=dict(thickness=20,len=0.3,x=0.27,y=0.9)),
                       showlegend=False)
fig.add_trace(trace_1, row=1, col=1)

## Input 2 ##
trace_2 = go.Scatter3d(x = over_pred['X'], y = over_pred['Y'], z = over_pred['Z'],mode='markers', 
                       marker=dict(colorscale='Viridis',size=8,line=dict(width=0),symbol='square',color=over_pred['Input 2'],colorbar=dict(thickness=20,len=0.3,x=0.62,y=0.9)),
                       showlegend=False)
fig.add_trace(trace_2, row=1, col=2)

## Input 3 ##
trace_3 = go.Scatter3d(x = over_pred['X'], y = over_pred['Y'], z = over_pred['Z'],mode='markers', 
                       marker=dict(colorscale='Viridis',size=8,line=dict(width=0),symbol='square',color=over_pred['Input 3'],colorbar=dict(thickness=20,len=0.3,x=1.0,y=0.9)),
                       showlegend=False)

fig.add_trace(trace_3, row=1, col=3)